In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
import seaborn as sns


In [ ]:
# Enhanced Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

training_set = train_datagen.flow_from_directory('SkinDiseaseDataset/train',
                                                 target_size=(128, 128),
                                                 batch_size=32,  # Adjusted batch size
                                                 class_mode='categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('SkinDiseaseDataset/val',
                                            target_size=(128, 128),
                                            batch_size=32,  # Adjusted batch size
                                            class_mode='categorical')

In [ ]:
# Load VGG16 model, pre-trained on ImageNet data
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [ ]:
# New model on top of VGG16
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(9, activation='softmax')(x) 

In [ ]:
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Training the Model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
history = model.fit(x=training_set, 
          validation_data=test_set, 
          epochs=20, 
          callbacks=[early_stopping, model_checkpoint])

In [ ]:
# Set the Seaborn style
sns.set(style='darkgrid', palette='deep', font_scale=1.2)

# Create a dark background
plt.style.use('dark_background')

# Plotting the Training History
plt.figure(figsize=(14, 10))

# Plot Training Loss with Validation Loss
plt.subplot(2, 2, 3)
sns.lineplot(x=range(1, len(history.history['loss']) + 1), y=history.history['loss'], label='Training Loss', linewidth=2, marker='o', markersize=8, color='cyan')
sns.lineplot(x=range(1, len(history.history['val_loss']) + 1), y=history.history['val_loss'], label='Validation Loss', linewidth=2, marker='o', markersize=8, color='magenta')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Plot Training Accuracy with Validation Accuracy
plt.subplot(2, 2, 4)
sns.lineplot(x=range(1, len(history.history['accuracy']) + 1), y=history.history['accuracy'], label='Training Accuracy', linewidth=2, marker='s', markersize=8, color='lime')
sns.lineplot(x=range(1, len(history.history['val_accuracy']) + 1), y=history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, marker='s', markersize=8, color='yellow')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:

# Confidence Level

# Load the trained model
cnn = load_model("best_model.keras")

# Image path
# image_path = 'SkinDiseaseDataset/.jpg'
image_path='gettyimages.jpg'

# Load the image with the target size
img = load_img(image_path, target_size=(128, 128))

# Convert the image to an array
img_array = img_to_array(img)

# Scale the image
img_array = img_array / 255.0

# Expand dimensions to match the model's input format
img_array = np.expand_dims(img_array, axis=0)

# Make a prediction
prediction = cnn.predict(img_array)

# Set a confidence threshold
confidence_threshold = 0.5

# Get the maximum confidence score
max_confidence = np.max(prediction)

# Output the predicted class if the confidence is above the threshold
if max_confidence >= confidence_threshold:
    predicted_class = np.argmax(prediction, axis=1)
    print(f"Predicted class: {predicted_class} with confidence: {max_confidence:.2f}")
else:
    print(f"No skin disease detected. with confidence: {max_confidence:.2f}")

if max_confidence >= confidence_threshold:
    predicted_class = np.argmax(prediction, axis=1)
    predicted_class_name = list(training_set.class_indices.keys())[predicted_class[0]]
    print(f"Predicted class: {predicted_class_name} with confidence: {max_confidence:.2f}")
else:
    print(f"No skin disease detected. with confidence: {max_confidence:.2f}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Generate predictions on the test set
predictions = model.predict(test_set)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get true labels
true_labels = test_set.classes

# Create confusion matrix
conf_mat = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=training_set.class_indices.keys(), yticklabels=training_set.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Display classification report
print(classification_report(true_labels, predicted_labels, target_names=training_set.class_indices.keys()))
